- weighting?

In [1]:
import glob
from tqdm import tqdm

In [2]:
import os
import pandas as pd
from choicemodels import MultinomialLogit, MultinomialLogitResults
from choicemodels.tools import MergedChoiceTable

## Load CHTS data 

In [3]:
data_dir = '/home/mgardner/chts-2012/data'
chts_persons = pd.read_csv(os.path.join(data_dir, 'Deliv_PER.csv'), low_memory=False)
chts_persons_lookup = pd.read_csv(os.path.join(data_dir, 'LookUp_PER.csv'))
chts_households = pd.read_csv(os.path.join(data_dir, 'Deliv_HH.csv'))
chts_households_lookup = pd.read_csv(os.path.join(data_dir, 'LookUp_Home.csv'))

/home/mgardner/anaconda3/envs/proba/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (25,27,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Get Bay Area Households

In [4]:
sfbay_counties = [1, 13, 41, 55, 75, 81, 85, 95, 97]

In [5]:
chts_persons = pd.merge(
    chts_persons.set_index(['SAMPN','PERNO']),
    chts_persons_lookup.set_index(['SAMPN','PERNO']),
    left_index=True, right_index=True,
    suffixes=('_persons', '_lookup')).reset_index()

chts_persons = pd.merge(
    chts_persons.set_index(['SAMPN']),
    chts_households_lookup.set_index(['SAMPN']),
    left_index=True, right_index=True).reset_index()

chts_persons = chts_persons[chts_persons['HCTFIP'].isin(sfbay_counties)].reset_index()

In [6]:
chts_persons.shape

(24030, 188)

## Get Discretionary Activity Coordinates

In [7]:
acts = pd.read_csv(os.path.join(data_dir, 'Deliv_ACTIVITY.csv'))

In [8]:
acts = pd.merge(chts_persons, acts, on=['SAMPN','PERNO'], suffixes=('_persons', '_acts'))

- 1= PERSONAL ACTIVITIES (SLEEPING, PERSONAL CARE, LEISURE, CHORES)
- 4=EXERCISE (WITH OR WITHOUT EQUIPMENT)/PLAYING SPORTS
- 7=USING COMPUTER/TELEPHONE/CELL OR SMART PHONE OR OTHER COMMUNICATIONS DEVICE FOR PERSONAL ACTIVITIES
- 13=NON-WORK RELATED ACTIVITIES (SOCIAL CLUBS, ETC)
- 14=EXERCISE/SPORTS
- 15=VOLUNTEER WORK/ACTIVITIES
- 23=DRIVE THROUGH MEALS (SNACKS, COFFEE, ETC.)
- 24=DRIVE THROUGH OTHER (ATM, BANK)
- 26=SERVICE PRIVATE VEHICLE (GAS, OIL, LUBE, REPAIRS)
- 27=ROUTINE SHOPPING (GROCERIES, CLOTHING, CONVENIENCE STORE, HH MAINTENANCE)
- 28=SHOPPING FOR MAJOR PURCHASES OR SPECIALTY ITEMS (APPLIANCE, ELECTRONICS, NEW VEHICLE, MAJORHH REPAIRS)
- 29=HOUSEHOLD ERRANDS (BANK, DRY CLEANING, ETC.)
- 30=PERSONAL BUSINESS (VISIT GOVERNMENT OFFICE, ATTORNEY, ACCOUNTANT)
- 31=EAT MEAL AT RESTAURANT/DINER
- 32=HEALTH CARE (DOCTOR, DENTIST, EYE CARE, CHIROPRACTOR, VETERINARIAN)
- 33=CIVIC/RELIGIOUS ACTIVITIES
- 34=OUTDOOR EXERCISE (PLAYING SPORTS/JOGGING, BICYCLING, WALKING, WALKING THE DOG, ETC.)
- 35=INDOOR EXERCISE (GYM, YOGA, ETC.)
- 36=ENTERTAINMENT (MOVIES, WATCH SPORTS, ETC)
- 37=SOCIAL/VISIT FRIENDS/RELATIVES

In [9]:
acts_disc = acts[acts['APURP'].isin([1, 4, 7, 13, 14, 15, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37])]

In [10]:
places = pd.read_csv(os.path.join(data_dir, 'Deliv_PLACE.csv'), low_memory=False)
places = places[places['STATE'] == 'CA']

In [11]:
places_lookup = pd.read_csv(os.path.join(data_dir, 'LookUp_PLACE.csv'), low_memory=False)

In [12]:
acts_places = pd.merge(
    acts_disc, places[[col for col in places if col not in ['PNAME',
 'CITY',
 'ZIP',
 'XCORD',
 'YCORD',
 'CTFIP',
 'TRACT',
 'BLOCK',
 'PPrimaryCity']]], on=['SAMPN', 'PERNO', 'PLANO'], suffixes=('_acts_disc', '_places'))

In [13]:
acts_places.shape

(88568, 249)

In [14]:
acts_places = pd.merge(acts_places, places_lookup, on=['SAMPN','PERNO', 'PLANO'], suffixes=('_acts_disc_places', '_lookup'))

In [15]:
acts_places = acts_places[acts_places['CTFIP'].isin(sfbay_counties)]

In [16]:
acts_places.shape

(86607, 258)

In [17]:
acts_places = acts_places[acts_places['PNAME'] != 'HOME']

In [18]:
acts_places.shape

(32810, 258)

In [19]:
acts_places[['PERWGT_persons', 'TCFPERWGT_places', 'TCFPERWGT_acts_disc']].describe()

,PERWGT_persons,TCFPERWGT_places,TCFPERWGT_acts_disc
count,32810.000000,32432.000000,32432.000000
mean,0.812878,0.923496,0.923496
std,0.651369,0.784478,0.784478
min,0.005002,0.004229,0.004229
25%,0.411571,0.446500,0.446500
50%,0.645009,0.709160,0.709160
75%,0.988743,1.131216,1.131216
max,5.060090,10.490937,10.490937


In [121]:
obs_df = pd.merge(
    acts_places[['SAMPN', 'AGE', 'EMPLY', 'HBLOCK', 'BLOCK']], chts_households[['SAMPN', 'INCOM', 'HHVEH', 'HHSIZ', 'HHLIC']], on='SAMPN')

In [122]:
obs_df.shape

(32810, 9)

In [123]:
obs_df['HBLOCK'] = obs_df['HBLOCK'].astype(str).str.zfill(15)
obs_df['BLOCK'] = obs_df['BLOCK'].astype(str).str.zfill(15)

## Get BEAM Block Group-level Skims

In [21]:
all_files = glob.glob("/home/mgardner/PILATES/pilates/beam/beam_output/result*")

In [54]:
chts_skims = pd.DataFrame()
for file in tqdm(all_files):
    skims = pd.read_csv(
        file, usecols=['timePeriod', 'pathType', 'origin', 'destination','DIST_meters'], low_memory=False)
    skims = skims[(skims['timePeriod'] == 'AM') & (skims['pathType'] == 'SOV')]
    chts_skims = pd.concat((chts_skims, skims), ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [05:51<00:00, 13.53s/it]


In [55]:
chts_skims.to_csv('../data/chts_skims.csv', index=False)

In [220]:
chts_skims = pd.read_csv('../data/chts_skims.csv')

## Create Alts from UrbanSim Block Data

In [198]:
store = pd.HDFStore('/home/mgardner/PILATES/pilates/urbansim/data/custom_mpo_06197001_model_data.h5')
blocks = store['blocks']

In [163]:
hh = store['households']

In [172]:
pp_block = hh.groupby('block_id')['persons'].sum()

In [126]:
jobs = store['jobs']

In [125]:
blocks.shape

(108469, 13)

In [127]:
jobs['sector_id'].unique()

array(['11', '21', '22', '23', '31-33', '42', '44-45', '48-49', '51',
       '52', '53', '54', '55', '56', '61', '62', '71', '72', '81', '92'],
      dtype=object)

In [128]:
retail_jobs = jobs[jobs['sector_id'] == '44-45'].groupby('block_id').count()

In [199]:
blocks = pd.merge(blocks, retail_jobs[['sector_id']], left_index=True, right_index=True, how='left').fillna(0)

In [200]:
blocks = pd.merge(blocks, pp_block, left_index=True, right_index=True, how='left').fillna(0)

In [201]:
blocks['pop_density'] = blocks['persons'] / blocks['square_meters_land']

In [204]:
blocks = blocks[blocks['square_meters_land'] > 0]

In [205]:
blocks = blocks[['sector_id', 'pop_density']]

In [206]:
blocks.columns = ['retail_jobs', 'pop_density']

In [207]:
blocks.shape

(106910, 2)

In [208]:
alts_df = blocks

In [209]:
alts_df['dest_block_grp'] = alts_df.index.str[0:12]

## Filter Choosers w/ Home Locations not in UrbanSim Block Data

In [133]:
obs_df = obs_df[obs_df['BLOCK'].isin(blocks.index)]

In [134]:
obs_df.shape

(32728, 9)

In [135]:
obs_df = obs_df[obs_df['HBLOCK'].isin(blocks.index)]

In [136]:
obs_df.shape

(32696, 9)

In [137]:
obs_df['block_grp'] = obs_df['HBLOCK'].str[0:12]

## MNL Feature Extraction

In [138]:
obs_df['inc_lt_75k'] = obs_df['INCOM'] < 6
obs_df['inc_gt_150k'] = obs_df['INCOM'] > 7
obs_df['inc_gt_200k'] = obs_df['INCOM'] > 8

In [139]:
obs_df['age_lt_18'] = obs_df['AGE'] < 18
obs_df['age_gt_65'] = obs_df['AGE'] > 65

In [155]:
obs_df['hhveh_gt_1'] = obs_df['HHVEH'] > 1
obs_df['hhveh_is_1'] = obs_df['HHVEH'] == 1
obs_df['hhveh_is_0'] = obs_df['HHVEH'] == 0
obs_df['hhveh_gt_0'] = obs_df['HHVEH'] > 0
obs_df['veh_pp'] = obs_df['HHVEH'] / obs_df['HHSIZ']
obs_df['veh_pp_gt_1'] = obs_df['veh_pp'] > 1
obs_df['veh_per_driver'] = obs_df['HHVEH'] / obs_df['HHLIC']
obs_df['veh_per_driver_gt_1'] = obs_df['veh_per_driver'] > 1

## Format Skims as Interaction Terms

In [230]:
chts_skims.rename(columns={'origin': 'block_grp', 'destination':'dest_block_grp'}, inplace=True)
chts_skims['block_grp'] = chts_skims['block_grp'].astype(str).str.zfill(12)
chts_skims['dest_block_grp'] = chts_skims['dest_block_grp'].astype(str).str.zfill(12)

#### Filter choosers/alts without origin/dest in skims

In [231]:
obs_df = obs_df[obs_df['block_grp'].isin(chts_skims['block_grp'])]
obs_df = obs_df[obs_df['BLOCK'].isin(alts_df.index)]

alts_df = alts_df[alts_df['dest_block_grp'].isin(chts_skims['dest_block_grp'])]

In [233]:
chts_skims.set_index(['block_grp', 'dest_block_grp'], inplace=True)

## Create Merged Choice Table

In [145]:
obs_df = obs_df[(obs_df['AGE'] < 98) & (obs_df['INCOM'] < 98) & (obs_df['HHSIZ'] < 98)]

In [234]:
%%time 
mct = MergedChoiceTable(
    obs_df, alts_df, chosen_alternatives='BLOCK', sample_size=1000, interaction_terms=chts_skims[['DIST_meters']])

CPU times: user 27.3 s, sys: 5.26 s, total: 32.6 s
Wall time: 32.6 s


In [235]:
mct_df = mct.to_frame().reset_index()

#### Impute missing dists

In [236]:
counts = mct_df.groupby('obs_id')[['DIST_meters']].count()
mean_dists = mct_df.groupby('obs_id')[['DIST_meters']].mean()

mct_df = pd.merge(mct_df, counts, left_on='obs_id', right_index=True)
mct_df = mct_df[mct_df['DIST_meters_y'] > 200]
mct_df = pd.merge(mct_df, mean_dists, left_on='obs_id', right_index=True)
mct_df['DIST_from_home'] = mct_df['DIST_meters_x'].copy()
mct_df.loc[pd.isnull(mct_df['DIST_meters_x']), 'DIST_from_home'] = mct_df.loc[pd.isnull(mct_df['DIST_meters_x']), 'DIST_meters']
mct_df = mct_df.sort_values(['obs_id', 'chosen'], ascending=False)
mct_df = mct_df[[
    'obs_id', 'block_id', 'age_lt_18', 'age_gt_65',
    'inc_lt_75k', 'inc_gt_150k', 'veh_pp','veh_per_driver', 'veh_pp_gt_1', 'veh_per_driver_gt_1',
    'hhveh_gt_1', 'hhveh_gt_0', 'hhveh_is_0', 'hhveh_is_1', 'retail_jobs','pop_density','DIST_from_home',
    'chosen']].set_index(['obs_id', 'block_id'])

In [219]:
mct_df[pd.isnull(mct_df['pop_density'])]

,,age_lt_18,age_gt_65,inc_lt_75k,inc_gt_150k,veh_pp,veh_per_driver,veh_pp_gt_1,veh_per_driver_gt_1,hhveh_gt_1,hhveh_gt_0,hhveh_is_0,hhveh_is_1,retail_jobs,pop_density,DIST_from_home,chosen
obs_id,block_id,,,,,,,,,,,,,,,,
28114,060855046011183,False,False,False,True,0.333333,0.500000,False,False,False,True,False,True,NaN,NaN,58503.149089,1
24515,060014511013308,False,False,False,False,0.666667,0.666667,False,False,True,True,False,False,NaN,NaN,70824.268802,1
10446,060855118003012,True,False,False,False,1.000000,1.000000,False,False,True,True,False,False,NaN,NaN,78604.878745,1
6657,060133500004032,False,False,False,True,1.000000,1.000000,False,False,True,True,False,False,NaN,NaN,58041.558345,1


In [237]:
mct = MergedChoiceTable.from_df(mct_df)

In [238]:
%%time
results = MultinomialLogit(
    mct,
    model_expression='np.log1p(retail_jobs) + np.log1p(pop_density) + (np.log(DIST_from_home) / (veh_pp_gt_1 + inc_lt_75k + inc_gt_150k + age_lt_18 + age_gt_65)) - 1')
results.fit()

CPU times: user 3min 58s, sys: 4min 56s, total: 8min 55s
Wall time: 57 s


                  CHOICEMODELS ESTIMATION RESULTS                   
Dep. Var.:                chosen   No. Observations:          28,783
Model:         Multinomial Logit   Df Residuals:              28,775
Method:       Maximum Likelihood   Df Model:                       8
Date:                 2021-12-24   Pseudo R-squ.:              0.438
Time:                      00:03   Pseudo R-bar-squ.:          0.437
AIC:                 223,688.832   Log-Likelihood:      -111,836.416
BIC:                 223,754.972   LL-Null:             -198,825.920
                                                 coef   std err            z     P>|z|   Conf. Int.
---------------------------------------------------------------------------------------------------
np.log1p(retail_jobs)                          0.7061     0.003      238.257     0.000             
np.log1p(pop_density)                        -33.4516     0.871      -38.393     0.000             
np.log(DIST_from_home)                        -1

In [107]:
%%time
results = MultinomialLogit(
    mct,
    model_expression='np.log1p(retail_jobs) + (np.log(DIST_from_home) / (hhveh_gt_1 + veh_pp + inc_lt_75k + inc_gt_150k + age_lt_18 + age_gt_65)) - 1')
results.fit()

CPU times: user 3min 39s, sys: 4min 29s, total: 8min 8s
Wall time: 52 s


                  CHOICEMODELS ESTIMATION RESULTS                   
Dep. Var.:                chosen   No. Observations:          28,787
Model:         Multinomial Logit   Df Residuals:              28,779
Method:       Maximum Likelihood   Df Model:                       8
Date:                 2021-12-23   Pseudo R-squ.:              0.438
Time:                      23:16   Pseudo R-bar-squ.:          0.437
AIC:                 223,710.714   Log-Likelihood:      -111,847.357
BIC:                 223,776.855   LL-Null:             -198,853.551
                                                coef   std err          z     P>|z|   Conf. Int.
------------------------------------------------------------------------------------------------
np.log1p(retail_jobs)                         0.7148     0.003    241.544     0.000             
np.log(DIST_from_home)                       -1.5654     0.002   -728.080     0.000             
np.log(DIST_from_home):hhveh_gt_1[T.True]    -0.0076     0.0